In [2]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from matplotlib import rcParams
import sympy as sy
import scipy.constants as pc
from sympy import *
from math import log10, floor

In [3]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)], zMaxArray[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (unsicherheiten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def einfachGaussMultMax(unsicherheiten, werte, exponenten):
    max_a = 0
    num = -1
    for i in range (unsicherheiten.size):
        maxi = (exponenten[i]*unsicherheiten[i]/werte[i])**2
        if max_a <= maxi:
            max_a = maxi
            num = i
    return (max_a, num)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]-(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

def round_to_j(x,j):
    return round(x, (j-1)-int(floor(log10(abs(x)))))


In [4]:
beta_rech = np.linspace(0.2, 5, 100)
gf_rech = 1+beta_rech**(-1)
bf_rech = beta_rech +1
ef_rech = 2+ beta_rech+ beta_rech**(-1)
#fig, ax = plt.subplots(figsize=(10,6))
#ax.tick_params(axis='both', which='major', labelsize=18)
#ax.tick_params(axis='both', which='minor', labelsize=16)
#ax.set_xscale('log')

#ax.plot(beta_rech, bf_rech, color='r', label=r'$\frac{b}{f}$')
#ax.plot(beta_rech, gf_rech, color='g', label=r'$\frac{g}{f}$ ')
#ax.plot(beta_rech, ef_rech, color='b', label=r'$\frac{e}{f}$ ')

#ax.set(xlabel=r'$\beta$', ylabel=r'Jeweilige Abhängigkeit', title=r'$\frac{b}{f}(\beta), \frac{g}{f}(\beta), \frac{e}{f}(\beta)$')
#ax.legend()
#ax.grid()

#fig.savefig("AbhBeta.svg")

In [5]:
#Gemmessene Werte
pos_dia = 200 #+- 1
e_gem_1 = np.array([647, 585, 552, 535, 522, 520, 521, 521, 525, 531, 536, 543, 550]) - pos_dia #sqrt(1^2+0.5^2)=sqrt(1.25)
g_gem_1 = np.array([300,310,320,330,340,350,360,370,380,390,400,410,420])#<-- nice +-1
b_gem_1 = e_gem_1-g_gem_1

print(e_gem_1)
print(g_gem_1)

e_gem_1_f, g_gem_f = np.sqrt(1.25), 1
b_gem_1_f = np.sqrt(e_gem_1_f**2+g_gem_f**2) #Ich hab das mit der Formewl gemacht, das geht so!

f_gem_1 = g_gem_1*b_gem_1/(g_gem_1+b_gem_1) #Brennweite

beta_gem_1 = b_gem_1/g_gem_1

#Nominelle Werte
f_nom_1 = 80 
bf_nom_1 = (1-(f_nom_1/g_gem_1))**(-1)
gf_nom_1 = (1-(f_nom_1/b_gem_1))**(-1)
ef_nom_1 = (((1/f_nom_1)-(1/g_gem_1))**(-1)+((1/f_nom_1)-(1/b_gem_1))**(-1))/(f_nom_1)


#fig, ax = plt.subplots(figsize=(10,6))
#ax.tick_params(axis='both', which='major', labelsize=18)
#ax.tick_params(axis='both', which='minor', labelsize=16)
#ax.set_xscale('log')

#ax.plot(x, y, color='b', label='label')
#ax.plot(beta_gem_1, bf_nom_1, color='r', label=r'$\frac{b}{f}$ nominell')
#ax.plot(beta_gem_1, gf_nom_1, color='g', label=r'$\frac{g}{f}$ nominell')
#ax.plot(beta_gem_1, ef_nom_1, color='b', label=r'$\frac{e}{f}$ nominell')
#ax.errorbar(am_x, am_y, xerr=std_x, yerr=std_y, fmt='D', color='#d0740b', label='label')

#ax.set(xlabel=r'$\beta$', ylabel=r'Jeweilige Abhängigkeit', title=r'$\frac{b}{f}(\beta), \frac{g}{f}(\beta), \frac{e}{f}(\beta)$')
#ax.legend()
#ax.grid()

#g, b, f = symbols('g b f')
#func = ((1/b)+(1/g))**(-1)
#print(f_gem_1)
#for i in range(len(b_gem_1)):
#    print(fehler(func, np.array([g_gem_1[i], b_gem_1[i]]), np.array([5, 5]), np.array([g, b])))

#fig.savefig("AbhängigkeitBeta.png")

[447 385 352 335 322 320 321 321 325 331 336 343 350]
[300 310 320 330 340 350 360 370 380 390 400 410 420]


In [5]:
# Erinnerungen
# x = np.array([,])
# x.size
# x, y = symbols('x y')
# expr = x*y
# expr.subs([(x, wert_x), (y, wert_y)])

In [7]:
#Gemessene Werte
g_gem_2 = np.array([300,310,320,330,340,350,360,370])  
e_gem_2_unv = np.array([432,426,427,430,435,440,446,452]) #Unvertauschte Linsen
e_gem_2_ver = np.array([450,431,425,424,430,435,440,445]) #Vertauschte Linsen

print(g_gem_2)
print(e_gem_2_unv)
print(e_gem_2_ver)




d_2_w, d_2_f = 20, 1

e_gem_2_f, g_gem_f, d_f = 2,0.5 , np.sqrt(0.1**2 + 1**2) #Ich hab das mit der Formewl gemacht, das geht so! copium
b_gem_2_f = np.sqrt(e_gem_2_f**2+g_gem_f**2) #Ich hab das mit der Formewl gemacht, das geht so!

b_gem_2_unv = e_gem_2_unv-g_gem_2
b_gem_2_ver = e_gem_2_ver-g_gem_2

f_gem_2_unv = g_gem_2*b_gem_2_unv/(g_gem_2+b_gem_2_unv)
f_fem_2_ver = g_gem_2*b_gem_2_ver/(g_gem_2+b_gem_2_ver)

#Nominalwerte
f_1, f_2 = 80, 150
f_nom_2_unv = (1/(f_1))+(1/(f_2))-d/(f_1+f_2)
f_nom_2_ver = (1/(f_2))+(1/(f_1))-d/(f_2+f_1) 
print(f_nom_2_unv, f_nom_2_ver) #Die beiden Werte sind gleich, daher kein unv und ver

f_1, f_2 = 80, 150
f_nom_2 = (1/(f_1))+(1/(f_2))-d/(f_1+f_2)

bf_nom_2 = (1-(f_nom_2/g_gem_2))**(-1)
#gf_nom_2 = (1-(f_nom_2/b_gem_2))**(-1) #Müsste hier nicht bf_nom_2 hin? Ansonsten braucht man das noch für unv und für ver

ef_nom_2 = (((1/f_nom_2)-(1/g_gem_2))**(-1)+((1/f_nom_2)-(1/bf_nom_2))**(-1))/(f_nom_2) #Hier musste glaube ich anstatt b_gem_2 >>> bf_nom_2 gemacht werden.
                                                                                    
#ef_nom_2_unv = (((1/f_nom_2)-(1/g_gem_2))**(-1)+((1/f_nom_2)-(1/b_gem_2_unv))**(-1))/(f_nom_2) #Wenn doch, dann das hier verwenden
#ef_nom_2_ver = (((1/f_nom_2)-(1/g_gem_2))**(-1)+((1/f_nom_2)-(1/b_gem_2_ver))**(-1))/(f_nom_2)

[300 310 320 330 340 350 360 370]
[432 426 427 430 435 440 446 452]
[450 431 425 424 430 435 440 445]


NameError: name 'd' is not defined

In [8]:
#Gemessene Werte


g_gem_3 = np.array([400,410,420,430,440,450,460,470])  
e_gem_3_unv = np.array([732,704,685,674,667,665,664,653]) #Unvertauschte Linsen
e_gem_3_ver = np.array([662,661,661,663,669,673,679,681]) #Vertauschte Linsen
Bg_gem_3_unv = np.array([15,12,11,10,8,8,7,6])
Bg_gem_3_ver = np.array([8,7,6,6,6,5,5,5])
print(g_gem_3)
print(e_gem_3_unv)
print(e_gem_3_ver)
print(Bg_gem_3_unv)
print(Bg_gem_3_ver)


[400 410 420 430 440 450 460 470]
[732 704 685 674 667 665 664 653]
[662 661 663 669 673 679 681]
[15 12 11 10  8  8  7  6]
[8 7 6 6 6 5 5 5]


In [ ]:
#Gemessene Werte 

f_4 = 250
f_gem_4_blau = 438
f_gem_4_rot = 446

